In [ ]:
import json
from io import StringIO
import numpy as np
import pandas as pd
from dotenv import dotenv_values
from pyexeggutor import (
    format_header,
    read_pickle,
)
import matplotlib.colors as mcolors
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go


from kegg_pathway_profiler.pathways import (
    pathway_coverage_wrapper,
)
from kegg_pathway_profiler.enrichment import (
    unweighted_pathway_enrichment_wrapper,
)

from nichespace.neighbors import (
    pairwise_distances_kneighbors,
    KNeighborsLeidenClustering,
)
from nichespace.manifold import (
    HierarchicalNicheSpace,
    QualitativeSpace,
    EmbeddingAnnotator,
    DEFAULT_REGRESSOR,
    DEFAULT_REGRESSOR_PARAM_SPACE,
)


from nichespace.llm import (
    LLMAnnotator,
)


In [ ]:
%%time
# Data
df_quality = pd.read_csv("../data/quality.tsv.gz",sep="\t", index_col=0)

quality_label="completeness_gte90.contamination_lt5"
# quality_label="completeness_gte50.contamination_lt10"

genome_to_clusterani = pd.read_csv(f"../data/training/{quality_label}/y.tsv.gz", sep="\t", index_col=0, header=None).iloc[:,0].astype("category")
X_genomic_traits = pd.read_csv(f"../data/training/{quality_label}/X.tsv.gz", sep="\t", index_col=0).astype(bool)
X_genomic_traits_clusterani = pd.read_csv(f"../data/training/{quality_label}/X_grouped.tsv.gz", sep="\t", index_col=0).astype(bool)

genome_to_taxonomy = pd.read_csv("../data/taxonomy.tsv.gz", sep="\t", index_col=0).iloc[:,0]
clusterani_to_taxonomy = pd.read_csv("../data/cluster/ani/cluster-ani_to_taxonomy.tsv.gz", sep="\t", index_col=0, header=None).iloc[:,0]
df_meta_genomes = pd.read_csv(f"../data/cluster/mfc/{quality_label}/identifier_mapping.mfc.genomes.with_openai.tsv.gz", sep="\t", index_col=0)
df_meta_genome_clusters = pd.read_csv(f"../data/cluster/mfc/{quality_label}/identifier_mapping.mfc.genome_clusters.with_openai.tsv.gz", sep="\t", index_col=0)

df_meta_keggortholog = pd.read_csv("/home/ec2-user/SageMaker/s3/newatlantis-raw-veba-db-prod/VEBA/VDB_v8.1/Annotate/KOfam/kegg-ortholog_metadata.tsv", sep="\t", index_col=0)
keggortholog_to_name = df_meta_keggortholog["definition"]
keggmodule_to_name = pd.read_csv("/home/ec2-user/SageMaker/s3/newatlantis-raw-veba-db-prod/VEBA/VDB_v8.1/Annotate/KEGG-Pathway-Profiler/pathway_names.tsv.gz", sep="\t", index_col=0, header=None).iloc[:,0]
kegg_pathway_database = read_pickle("/home/ec2-user/SageMaker/s3/newatlantis-raw-veba-db-prod/VEBA/VDB_v8.1/Annotate/KEGG-Pathway-Profiler/database.pkl.gz")
print("Number of genomes: {}, Number of features: {}, Number of SLCs: {}".format(*X_genomic_traits.shape, X_genomic_traits_clusterani.shape[0]))
# Number of genomes: 20377, Number of features: 2124, Number of SLCs: 6719



In [ ]:
# Clustering 
clusterer = read_pickle(f"../data/cluster/mfc/completeness_gte90.contamination_lt5/MFC.PathwayKOfam.KNeighborsLeidenClustering.pkl")
clusterer

In [ ]:
# MNS
mns = read_pickle(f"../data/training/completeness_gte90.contamination_lt5/NAL-GDB_MNS_v2.SLC-MFC.HierarchicalNicheSpace.pkl")
mns

In [ ]:
# Qualitative MNS
qualitative_mns = read_pickle(f"../data/training/completeness_gte90.contamination_lt5/NAL-GDB_MNS_v2.SLC-MFC.QualitativeSpace.pkl")
qualitative_mns

In [ ]:
# Embedding Annnotator
annotator = read_pickle(f"../data/training/completeness_gte90.contamination_lt5/NAL-GDB_MNS_v2.SLC-MFC.EmbeddingAnnotator.pkl")
annotator

In [ ]:
# Load KEGG module database

# # Calculate KEGG module completion ratios
# dataframes = list()
# for id_niche, features in annotator.selected_features_.items():
#     data = pathway_coverage_wrapper(
#         evaluation_kos=set(features), # Annotate just the first niche
#         database=kegg_pathway_database,
#     )
#     df = pd.DataFrame(data).T
#     df.insert(0, "name", df.index.map(lambda x: keggmodule_to_name[x]))
#     df.index = df.index.map(lambda x: (id_niche, x))
#     dataframes.append(df)
     
# df_kegg_coverage = pd.concat(dataframes, axis=0)
# df_kegg_coverage.to_csv("../data/training/completeness_gte90.contamination_lt5/NAL-GDB_MNS_v2.SLC-MFC.EmbeddingAnnotator.kegg_module_coverage.tsv.gz", sep="\t")
df_kegg_coverage = pd.read_csv("../data/training/completeness_gte90.contamination_lt5/NAL-GDB_MNS_v2.SLC-MFC.EmbeddingAnnotator.kegg_module_coverage.tsv.gz", sep="\t", index_col=[0,1])

# # Calculate KEGG module enrichment
# dataframes = list()
# for id_niche, features in annotator.selected_features_.items():
#     df = unweighted_pathway_enrichment_wrapper(
#     evaluation_kos=set(features), 
#     database=kegg_pathway_database,
#     background_set=set(mns.X_.columns),
#     )
#     df.insert(0, "name", df.index.map(lambda x: keggmodule_to_name[x]))
#     df.index = df.index.map(lambda x: (id_niche, x))
#     dataframes.append(df)
     
# df_kegg_enrichment = pd.concat(dataframes, axis=0)
# df_kegg_enrichment.to_csv("../data/training/completeness_gte90.contamination_lt5/NAL-GDB_MNS_v2.SLC-MFC.EmbeddingAnnotator.kegg_module_enrichment.tsv.gz", sep="\t")
df_kegg_enrichment = pd.read_csv("../data/training/completeness_gte90.contamination_lt5/NAL-GDB_MNS_v2.SLC-MFC.EmbeddingAnnotator.kegg_module_enrichment.tsv.gz", sep="\t", index_col=[0,1])


In [ ]:
# Merge genomes and ANI-cluster seed observations
df_meta_scatter = df_meta_genome_clusters.copy()
df_meta_scatter["id_cluster-ani"] = pd.NA
df_meta_scatter = pd.concat([
    df_meta_scatter,
    df_meta_genomes,
], axis=0)

                
# Sample data based on your provided DataFrame
df_scatter = pd.concat([
    qualitative_mns.embedding_,
    df_meta_scatter,
], axis=1)
df_scatter["id_observation"] = df_scatter.index
df_scatter["type"] = df_scatter["database"].map(lambda x: {True:"ANI-Cluster", False:"Genome"}[pd.isnull(x)])


# Generate 65 colors with custom lightness and saturation.
colors = sns.hls_palette(65, l=0.6, s=0.9)

# Get unique classes from your DataFrame.
unique_classes = df_scatter["id_cluster-mfc"].dropna().unique()

# Create a color dictionary mapping each unique class to a hex color.
class_to_color = {cat: mcolors.to_hex(color) for cat, color in zip(unique_classes, colors)}

font_color = "rgb(211,255,34)"
background_color = "rgb(15,20,40)"
grid_color = "rgb(24,30,64)"

# Create the 3D scatter plot with additional hover fields.
fig = px.scatter_3d(
    df_scatter,
    x="PaCMAP-1",
    y="PaCMAP-2",
    z="PaCMAP-3",
    hover_name="id_observation",
    color="id_cluster-mfc",  # color points based on this column
    color_discrete_map=class_to_color,  # use your custom mapping
    hover_data=[
        "id_cluster-ani",
        "taxonomy",
        "id_cluster-mfc",
        "database",
        "genome_size",
        "domain",
        "number_of_genes",
        "coding_size",
        "completeness",
        "contamination",
        "openai|gpt-4o-mini|enriched-taxa",
        "openai|gpt-4o-mini|metabolism-description",
        "openai|gpt-4o-mini|ecological-description"
    ],
    title=qualitative_mns.name,

)


fig.update_layout(
    height=800,
    width=1500,
    paper_bgcolor=background_color,  # Background color outside the plotting area
    scene=dict(
        bgcolor=background_color,    # 3D plot background color
        xaxis=dict(
            backgroundcolor=background_color,
            gridcolor=grid_color,
            color=font_color
        ),
        yaxis=dict(
            backgroundcolor=background_color,
            gridcolor=grid_color,
            color=font_color
        ),
        zaxis=dict(
            backgroundcolor=background_color,
            gridcolor=grid_color,
            color=font_color
        )
    ),
    font=dict(color=font_color)
)
fig.update_traces(marker=dict(size=5))
for trace in fig.data:
    trace.marker.line = dict(color=grid_color, width=0.382)
fig.show()

In [ ]:
# Merge genomes and ANI-cluster seed observations
def query(key, df):
    a = df["openai|gpt-4o-mini|metabolism-description"].map(lambda x: key.lower() in str(x))
    b = df["openai|gpt-4o-mini|ecological-description"].map(lambda x: key.lower() in str(x))
    return (a + b) > 0

df_meta_scatter = df_meta_genome_clusters.copy()
df_meta_scatter["id_cluster-ani"] = pd.NA
df_meta_scatter = pd.concat([
    df_meta_scatter,
    df_meta_genomes,
], axis=0)

                
# Sample data based on your provided DataFrame
df_scatter = pd.concat([
    qualitative_mns.embedding_,
    df_meta_scatter,
], axis=1)
df_scatter["id_observation"] = df_scatter.index
df_scatter["type"] = df_scatter["database"].map(lambda x: {True:"ANI-Cluster", False:"Genome"}[pd.isnull(x)])
df_scatter["category[methane_cycle]"] = query("methan", df_scatter)



# Create a color dictionary mapping each unique class to a hex color.
class_to_color = {True:"red", False:"white"}

font_color = "rgb(211,255,34)"
background_color = "rgb(15,20,40)"
grid_color = "rgb(24,30,64)"

# Create the 3D scatter plot with additional hover fields.
fig = px.scatter_3d(
    df_scatter,
    x="PaCMAP-1",
    y="PaCMAP-2",
    z="PaCMAP-3",
    hover_name="id_observation",
    color="category[methane_cycle]",  # color points based on this column
    color_discrete_map=class_to_color,  # use your custom mapping
    hover_data=[
        "id_cluster-ani",
        "taxonomy",
        "id_cluster-mfc",
        "database",
        "genome_size",
        "domain",
        "number_of_genes",
        "coding_size",
        "completeness",
        "contamination",
        "openai|gpt-4o-mini|enriched-taxa",
        "openai|gpt-4o-mini|metabolism-description",
        "openai|gpt-4o-mini|ecological-description"
    ],
    title=qualitative_mns.name,

)


fig.update_layout(
    height=800,
    width=1500,

    paper_bgcolor=background_color,  # Background color outside the plotting area
    scene=dict(
        bgcolor=background_color,    # 3D plot background color
        xaxis=dict(
            backgroundcolor=background_color,
            gridcolor=grid_color,
            color=font_color
        ),
        yaxis=dict(
            backgroundcolor=background_color,
            gridcolor=grid_color,
            color=font_color
        ),
        zaxis=dict(
            backgroundcolor=background_color,
            gridcolor=grid_color,
            color=font_color
        )
    ),
    font=dict(color=font_color)
)
fig.update_traces(marker=dict(size=5))
for trace in fig.data:
    trace.marker.line = dict(color=grid_color, width=0.382)
fig.show()

In [ ]:

# Define your color variables
font_color = "rgb(211,255,34)"
background_color = "rgb(15,20,40)"
grid_color = "rgb(24,30,64)"

# Extract data for the query (assuming mns.diffusion_coordinates_initial_ is a DataFrame or Series)
query = "XB_MAG545"
data = mns.diffusion_coordinates_initial_.loc[query]

# Convert the Series into a DataFrame:
# The index represents the metabolic niche names and the values are the diffusion strengths.
df_bar = data.reset_index()
df_bar.columns = ["Metabolic Niche", "Diffusion Strength"]

# Create the bar chart using Plotly Express
fig = px.bar(df_bar, x="Metabolic Niche", y="Diffusion Strength")

# Add a horizontal line at y=0
fig.add_hline(y=0, line_dash="dash", line_color=grid_color)

# Update the layout: set the background colors and font color.
fig.update_layout(
    height=700,
    width=1200,
    paper_bgcolor=background_color,  # color outside the plot area
    plot_bgcolor=background_color,   # color inside the plot area
    font=dict(color=font_color),
)

# Set the bar color to be font_color
fig.update_traces(marker_color=font_color)

# Update the axis labels and tick colors
fig.update_xaxes(title_text="Metabolic Niche", color=font_color, gridcolor=grid_color)
fig.update_yaxes(title_text="Diffusion Strength", color=font_color, gridcolor=grid_color)

fig.show()


In [ ]:
import pandas as pd
import plotly.express as px

# Assume you have your series defined as follows:
data = annotator.feature_weights_["n35"].sort_values(ascending=False)
data.index = data.index.map(lambda x: f"{x} | {keggortholog_to_name[x]}")

# Convert the Series into a DataFrame
df = data.reset_index()
df.columns = ["Feature", "Weight"]

# Create a horizontal bar chart (set orientation to "h")
fig = px.bar(
    df,
    x="Weight",
    y="Feature",
    orientation="h",
    title="Feature Weights for n35"
)

# Define your color variables
font_color = "rgb(211,255,34)"
background_color = "rgb(15,20,40)"

# Update the layout: set the background colors and font color.
fig.update_layout(
    height=700,
    width=1200,
    paper_bgcolor=background_color,  # color outside the plot area
    plot_bgcolor=background_color,   # color inside the plot area
    font=dict(color=font_color),
)

# Set the bar color to be font_color
fig.update_traces(marker_color=font_color)

fig.show()


In [ ]:
best_iteration = pd.DataFrame(annotator.automl_results_["n35"]).sort_values(["feature_selected_testing_score", "feature_selected_training_score"], ascending=[False, False]).iloc[0]
estimator = best_iteration["best_estimator"]
# Compute actual and predicted values
y_training = annotator.Y_["n35"]
y_training_hat = pd.Series(
    estimator.predict(annotator.X_.loc[:, estimator.feature_names_in_]),
    index=annotator.X_.index,
)

y_testing = annotator.Y_testing_["n35"]
y_testing_hat = pd.Series(
    estimator.predict(annotator.X_testing_.loc[:, estimator.feature_names_in_]),
    index=annotator.X_testing_.index,
)

# Combine the training and testing data
y = pd.concat([y_training, y_testing])
y_hat = pd.concat([y_training_hat, y_testing_hat])

# Create a DataFrame for plotting. We label points as "Training" or "Testing".
df_scatter = pd.DataFrame({
    "Actual": y,
    "Predicted": y_hat,
    "Group": ["Training"] * y_training.size + ["Testing"] * y_testing.size
})

# Create a scatter plot with custom colors for each group.
fig = px.scatter(
    df_scatter,
    x="Actual",
    y="Predicted",
    color="Group",
    color_discrete_map={"Training": "white", "Testing": font_color},
    title="Actual vs Predicted for n35"
)

# Define your color scheme
font_color = "rgb(211,255,34)"
background_color = "rgb(15,20,40)"
grid_color = "rgb(24,30,64)"

# Update the layout with your color theme
fig.update_layout(
    height=700,
    width=1200,
    paper_bgcolor=background_color,
    plot_bgcolor=background_color,
    font=dict(color=font_color)
)
fig.update_xaxes(gridcolor=grid_color, color=font_color)
fig.update_yaxes(gridcolor=grid_color, color=font_color)
fig.update_traces(marker=dict(size=5))
for trace in fig.data:
    trace.marker.line = dict(color=grid_color, width=0.382)
fig.show()


In [ ]:
list(map(lambda x: (x, keggortholog_to_name[x]), kegg_pathway_database["M00175"]["ko_to_nodes"].keys()))

In [ ]:
# Load credentials
config = dotenv_values("/home/ec2-user/SageMaker/.openai")

# Setup client
llm = LLMAnnotator(**config, description="EmbeddingAnnotator Contextualizer")
llm

In [ ]:
prompts = dict()
prompts["annotate_embeddings[kegg_orthogs=True,genome_taxonomy=False]"] = """
You are an AI assistant tasked with analyzing a dictionary of KEGG orthologs (features) and their corresponding weights. 
These weights indicate the predictive capacity of each ortholog for a specific dimension in a diffusion map embedding. 
Your goal is to assign high-level metabolic or ecological annotations to this embedding dimension, providing insight into
the metabolic niche of organisms with high magnitude along this dimension.

Here is the dictionary of features (KEGG orthologs) and their weights:
<feature_weight_dict>
%s
</feature_weight_dict>

To complete this task, follow these steps:
1. Examine the features and weights in the dictionary. Pay attention to the magnitude and sign of the weights. 
Higher magnitude will mean these proteins will be the most influential in predicting the diffusion coordinate
while the sign indicates a separation along some data-driven axis likely representing complex metabolic patterns.
2. Use your knowledge of KEGG pathways and biological processes to understand the function of these orthologs 
and their roles in metabolic pathways.
3. Contextualize your research by searching for common themes or related processes among the weighted features.
4. Based on your analysis, formulate a high-level metabolic or ecological annotation for this embedding dimension. 
Consider what type of organisms or metabolic processes might be associated with high magnitude along this dimension.
Provide your answer in json format that be read into a Python dictionary without additional parsing:
{
"summary":[Here, provide a consise analysis of your findings in and how the features relate to each other metabolically and ecologically.],
"metabolic_context":[Provide a concise, high-level annotation of the metabolic context for the embedding dimension based on your analysis.],
"ecological_context":[Provide a concise, high-level annotation of the ecological context for the embedding dimension based on your analysis.]
"justification":[Explain your reasoning for the annotation, citing specific features and their weights from the dictionary. Discuss any potential limitations or uncertainties in your interpretation.]
}
Remember to think critically about the biological significance of these features and their weights. Consider how they might work together in metabolic pathways or ecological processes. 
If you're unsure about certain aspects, acknowledge this uncertainty in your justification.
Your goal is to provide a well-reasoned, biologically meaningful annotation that could help researchers understand the metabolic or ecological significance of this embedding dimension.
"""

In [ ]:
info = annotator.feature_weights_["n35"].sort_values(ascending=False)
info.index = info.index.map(lambda x: (x,keggortholog_to_name[x]))
info

In [ ]:
prompt = prompts["annotate_embeddings[kegg_orthogs=True,genome_taxonomy=False]"]%(str(info.to_dict()))
response = llm.query(prompt)

In [ ]:
d = json.load(StringIO(response))
for k, v in d.items():
    print(format_header(k))
    print(v)
    print()